In [6]:
import cv2
import os.path as osp
import os
import glob
import numpy as np
import shutil
from Pose.Hrnet import Hrnet
from Pose.Yolov7 import Yolov7
import mmcv.utils.progressbar as progressbar
import mmcv

In [2]:
pose_model = Hrnet(engine_path='Pose/Hrnet48_fp32.trt')
pose_model.get_fps()
pose_model.destory()
det_model = Yolov7(engine_path='Pose/yolov7_fp16.trt')
det_model.get_fps()
det_model.destory()

91.661548893398 FPS
154.67425601459598 FPS


In [3]:
skeleton_edge = [(15, 13), (13, 11), (16, 14), (14, 12), (11, 12),
                                (5, 11), (6, 12), (5, 6), (5, 7), (6, 8), (7, 9),
                                (8, 10), (1, 2), (0, 1), (0, 2), (1, 3), (2, 4),
                                (3, 5), (4, 6)]

def inference_image(img,detect:Yolov7,pose:Hrnet):
    det_results = detect.inference(img)
    pose_results = pose.inference_from_bbox(img,det_results)
    return pose_results



def vis_pose(image, pose_result,threshold = 0.5):
        bbox = []
        bbox_score = []
        keypoints = []
        keypoints_score = []
        if pose_result is None:
            return image
        for pos in pose_result:
            bbox.append(pos['bbox'][:4])
            bbox_score.append(pos['bbox'][4])
            keypoints.append(pos['keypoints'][:,:2])
            keypoints_score.append(pos['keypoints'][:,2])
        # max_score_indx = np.argmax(bbox_score)
        # bbox = bbox[max_score_indx]
        # keypoints = keypoints[max_score_indx]
        # skeleton_features = pose_result[max_score_indx]['keypoints']
        # keypoints = keypoints
        # for edge in skeleton_edge:
        #     start = keypoints[edge[0]]
        #     end = keypoints[edge[1]]
        #     image = cv2.line(image, (int(start[0]), int(start[1])), (int(end[0]), int(end[1])), (255,255,0), 2)
        # for i in range(17):
        #     (x, y) = keypoints[i]
        # #     if self.label[i] == 0:
        # #         color = (255, 255, 255)
        # #     elif self.label[i] == 1:
        # #         color = (0, 0, 255)
        # #     elif self.label[i] == 2:
        # #         color = (255, 0, 0)
        #     image = cv2.circle(image, (int(x), int(y)), 4, (255, 255, 255), -1)

        # image = cv2.rectangle(image, (int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])) , (0,255,0), 1)
        # return image
        max_score_indx = np.argmax(bbox_score)
        bbox = bbox[max_score_indx]
        keypoints = keypoints[max_score_indx]
        keypoints_score = keypoints_score[max_score_indx]
        skeleton_features = pose_result[max_score_indx]['keypoints']
        keypoints = keypoints
        for edge in skeleton_edge:
            start = keypoints[edge[0]]
            end = keypoints[edge[1]]
            # image = cv2.line(image, (int(start[0]), int(start[1])), (int(end[0]), int(end[1])), (255,255,0), 2)
            if keypoints_score[edge[0]] < threshold or keypoints_score[edge[1]] < threshold:
                continue
            image = cv2.line(image, (int(start[0]), int(start[1])), (int(end[0]), int(end[1])), (255, 255, 0), 2)
        for i in range(17):
            (x, y) = keypoints[i]
            if keypoints_score[i] < threshold:
                continue
            image = cv2.circle(image, (int(x), int(y)), 4, (255, 255, 255), -1)

        image = cv2.rectangle(image, (int(bbox[0]), int(bbox[1])),(int(bbox[2]), int(bbox[3])) , (0,255,0), 1)
        return image

def extract_frame(video_path):
    dname = 'temp'
    os.makedirs(dname, exist_ok=True)
    frame_tmpl = osp.join(dname, 'img_{:05d}.jpg')
    cap = cv2.VideoCapture(video_path)
    frame_paths = []
    cnt = 0
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag:
            frame_path = frame_tmpl.format(cnt + 1)
            frame_paths.append(frame_path)
            frame=cv2.resize(frame,(640,480))
            cv2.imwrite(frame_path, frame)
            cnt += 1
        else: break
    cap.release()
    return frame_paths

def detection_inference(det_model:Yolov7,frame_paths,det_score=0.5):
    results = []
    print('Performing Human Detection for each frame')
    prog_bar = progressbar.ProgressBar(len(frame_paths))
    for frame_path in frame_paths:
        img = cv2.imread(frame_path)
        result = det_model.inference(img,det_score)
        # We only keep human detections with score larger than det_score_thr
        if len(result[2]) == 0:
            results.append(result)
            prog_bar.update()
            continue
        person_id = result[2] == 0
        bbox = result[0][person_id]
        score = result[1][person_id]
        indx = result[2][result[2]==0]
        results.append((bbox,score,indx))
        prog_bar.update()
    return results

def pose_inference(pose_model:Hrnet,frame_paths,det_results):
    print('Performing Human Pose Estimation for each frame')
    prog_bar = progressbar.ProgressBar(len(frame_paths))
    num_frame = len(det_results)
    num_person = max([len(x[2]) for x in det_results])
    if num_person == 0:
        kp = np.zeros((1,num_frame,17,3),dtype=np.float32)
        return kp
    kp = np.zeros((num_person,num_frame,17,3))
    for i ,(f,d) in enumerate(zip(frame_paths,det_results)):
        img = cv2.imread(f)
        pose_result = pose_model.inference_from_bbox(img,d)
        if pose_result is None:
            for person_id in range(num_person):
                kp[person_id,i] = kp[person_id,i-1]
        else:
            for j,item in enumerate(pose_result):
                kp [j,i] = item["keypoints"]            
        # if len(d[2]) == 0:
        #     for person_id in range(num_person):
        #         kp[person_id,i] = kp[person_id,i-1]
        #     prog_bar.update()
        #     continue
        vis_image = vis_pose(img,pose_result)
        cv2.imshow('',vis_image)
        if cv2.waitKey(20)& 0xFF==ord('q'): break
        prog_bar.update()
    cv2.destroyAllWindows()
    return kp

def pose_extraction(vid,label,pose_model:Hrnet=pose_model,det_model:Yolov7=det_model,det_score=0.5):
    frame_paths = extract_frame(vid)
    det_results = detection_inference(det_model,frame_paths,det_score)
    img = cv2.imread(frame_paths[0])
    img_shape = (img.shape[1],img.shape[0])
    pose_results = pose_inference(pose_model,frame_paths,det_results)
    anno = dict()
    anno['kp'] = pose_results
    anno['img_shape'] = img_shape
    anno['total_frames'] = pose_results.shape[1]
    anno['label'] = label
    shutil.rmtree(osp.dirname(frame_paths[0]))
    return anno
    

In [4]:
listfiles = []
listfiles.append(glob.glob('TRAIN/RUN/*.mp4'))
listfiles.append(glob.glob('TRAIN/WALK/*.mp4'))
listfiles.append(glob.glob('TRAIN/FALL/*.mp4'))
listfiles.append(glob.glob('TRAIN/SIT DOWN/*.mp4'))

In [5]:
anno_train = []
for index,files in enumerate(listfiles):
    for file in files:
        print('Processing ' + file)
        anno = pose_extraction(file,index,det_score=0.5) #LABEL NOT_FALL
        anno_train.append(anno)

Processing TRAIN/RUN\01.mp4
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 40/40, 55.8 task/s, elapsed: 1s, ETA:     0sPerforming Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 40/40, 20.6 task/s, elapsed: 2s, ETA:     0sProcessing TRAIN/RUN\02.mp4
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 42/42, 48.0 task/s, elapsed: 1s, ETA:     0sPerforming Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 42/42, 21.2 task/s, elapsed: 2s, ETA:     0sProcessing TRAIN/RUN\03.mp4
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 42/42, 57.7 task/s, elapsed: 1s, ETA:     0sPerforming Human Pose Estimation for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 42/42, 21.2 task/s, elapsed: 2s, ETA:     0sProcessing TRAIN/RUN\04.mp4
Performing Human Detection for each frame
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 42/42, 56.7 task/s, elapsed: 1s, ETA:     0sPerforming Human Pose E

In [8]:
import mmengine.fileio as fileio
fileio.dump(anno_train,'Data/Run_Walk_Fall_Sit.pkl')

In [7]:
import pandas as pd
pd.to_pickle(anno_train,'Data/pickle_file/Run_Walk_Fall_Sit.pkl')

In [15]:
anno2 = pd.read_pickle('Data/pickle_file/Run_Walk_Fall_Sit.pkl')

In [17]:
len(anno2)

496

In [ ]:

# files=glob.glob('VIDEO_TEST/*.mp4')
# file='VIDEO_TEST/VIDEO11.mp4'
lstfile = []
# lstfile.append(sorted(glob.glob('TRAIN/NOT FALL/*.mp4'),key=os.path.getmtime))
lstfile.append(sorted(glob.glob('TRAIN/FALL/*.mp4'),key=os.path.getmtime))
# lstfile.append(sorted(glob.glob('Video_Lying/*.mp4'),key=os.path.getmtime))
# lstfile=sorted(key=os.path.getmtime)
anno_train = []
for index,files in enumerate(lstfile):
    for file in files:
        print('Processing ' + file)
        anno = pose_extraction(file,1,det_score=0.8) #LABEL NOT_FALL
        anno_train.append(anno)

In [ ]:
# mmcv.dump(anno,'Data/Fall.pkl')

In [ ]:
# file='C:/Users/PhuTuan/Downloads/Video/Data_fall_171.mp4'
# # anno = pose_extraction(file,0,det_score=0.8)

In [ ]:
# frame_paths = extract_frame(file)
# det_results = detection_inference(yolov7,frame_paths,0.8)
# img = cv2.imread(frame_paths[0])
# img_shape = (img.shape[1],img.shape[0])

In [ ]:
# pose_results = pose_inference(hrnet,frame_paths,det_results)

In [ ]:
# anno = dict()
# anno['kp'] = pose_results
# anno['img_shape'] = img_shape
# anno['total_frames'] = pose_results.shape[1]

In [ ]:
# det_results[70]

In [ ]:
# frame_paths = extract_frame(file)
# det_results = detection_inference(yolov7,frame_paths,0.8)
# img = cv2.imread(frame_paths[0])
# img_shape = (img.shape[1],img.shape[0])
# pose_results = pose_inference(hrnet,frame_paths,det_results)
# anno = dict()
# anno['kp'] = pose_results
# anno['img_shape'] = img_shape
# anno['total_frames'] = pose_results.shape[1]

In [ ]:
# img = cv2.imread(frame_paths[71])
# result = yolov7.inference(img,0.8)

In [ ]:
# result[2] == 0